# Tavily Search

[Tavily's Search API](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package itself.

```bash
pip install -U langchain-community tavily-python
```

We also need to set our Tavily API key.

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

## Usage

We show here how to use it as part of an [agent](/docs/modules/agents). We use the OpenAI Functions Agent, so we will need to setup and install the required dependencies for that. We will also use [LangSmith Hub](https://smith.langchain.com/hub) to pull the prompt from, so we will need to install that.

```bash
pip install -U openai langchainhub
```

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.chat_models import ChatOpenAI
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(temperature=0)
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)
tools = [tavily_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

NameError: name 'TavilySearchAPIWrapper' is not defined

In [ ]:
agent_executor.run(
    "What happened in the latest burning man floods?",
)